# Schedule Access Plan Recommendation

This notebook is dedicated to model fitting in terms of database access plans.

In [56]:
# pandas
import pandas as pd
print('pandas: %s' % pd.__version__)
# numpy
import numpy as np
print('numpy: %s' % np.__version__)
# sklearn
import sklearn as sk
from sklearn import preprocessing
print('sklearn: %s' % sk.__version__)

pandas: 0.23.4
numpy: 1.15.2
sklearn: 0.18.1


In [57]:
#
# Experiment Config
tpcds='TPCDS1' # Schema upon which to operate test

### Read data from file into pandas dataframes

In [66]:
rep_vsql_plan_path = 'C:/Users/gabriel.sammut/University/Data_ICS5200/Schedule/' + tpcds + '/v2/rep_vsql_plan.csv'
#rep_vsql_plan_path = 'D:/Projects/Datagenerated_ICS5200/Schedule/' + tpcds + '/v2/rep_vsql_plan.csv'
#
rep_vsql_plan_df = pd.read_csv(rep_vsql_plan_path,sep='\t')
#
def prettify_header(headers):
    """
    Cleans header list from unwated character strings
    """
    header_list = []
    [header_list.append(header.replace("(","").replace(")","").replace("'","").replace(",","")) for header in headers]
    print(type(header_list))
    print(header_list)
    return header_list
#
#rep_vsql_plan_df.columns = prettify_header(rep_vsql_plan_df.columns.values)
print('------------------------------------------')
print(rep_vsql_plan_df.columns)
print(rep_vsql_plan_df['SQL_TEXT'].head())

------------------------------------------
Index(['('SQL_TEXT', 'SQL_ID', 'PLAN_HASH_VALUE', 'ID', 'OPERATION', 'OPTIONS', 'OBJECT_NODE', 'OBJECT#', 'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS', 'OBJECT_TYPE', 'OPTIMIZER', 'PARENT_ID', 'DEPTH', 'POSITION', 'SEARCH_COLUMNS', 'COST', 'CARDINALITY', 'BYTES', 'PARTITION_START', 'PARTITION_STOP', 'PARTITION_ID', 'DISTRIBUTION', 'CPU_COST', 'IO_COST', 'TEMP_SPACE', 'ACCESS_PREDICATES', 'FILTER_PREDICATES', 'PROJECTION', 'TIME', 'QBLOCK_NAME', 'TIMESTAMP')'], dtype='object')


KeyError: 'SQL_ID'

### Dealing with empty values

In [ ]:
def get_na_columns(df, headers):
    """
    Return columns which consist of NAN values
    """
    na_list = []
    for head in headers:
        if df[head].isnull().values.any():
            na_list.append(head)
    return na_list
#
print('N/A Columns\n')
print('\nREP_VSQL_PLAN Features ' + str(len(rep_vsql_plan_df.columns)) + ': ' + str(get_na_columns(df=rep_vsql_plan_df,headers=rep_vsql_plan_df.columns)) + "\n")
#
def fill_na(df):
    """
    Replaces NA columns with 0s
    """
    return df.fillna(0)
#
# Populating NaN values with amount '0'
df = fill_na(df=rep_vsql_plan_df)

### TPC-DS Filtering & Merging

Exclude all queries but those pertaining to TPC-DS.
This part also merges the field 'SQL_TEXT' from data matrix REP_HIST_SNAPSHOT with data matrix 'REP_VSQL_PLAN'.

In [ ]:
# #
# # Creating a dataframe consisting of a mapping library, composed of SQL_IDs & SQL_TEXTs
# sql_id_map = rep_hist_snapshot_df[['SQL_ID','SQL_TEXT']]
# print(sql_id_map.shape)
# #
# # Drop duplicate combinations from map
# sql_id_map.drop_duplicates(keep = False,
#                            inplace = True)
# print(sql_id_map.shape)
# #
# # Merge SQL_TEXT with Access Plan table
# df = pd.merge(sql_id_map, rep_vsql_plan_df, how='inner', on=['SQL_ID'])
# print('Plan table shape: ' + str(rep_vsql_plan_df.shape))
# print("Merged matrix shape: " + str(df.shape) + "\n\n--------------------------------------------------------\n\n")
# print(df.head())

### Feature Selection

In this step, redundant features are dropped. Features are considered redundant if exhibit a standard devaition of 0 (meaning no change in value).

In [ ]:
def drop_flatline_columns(df):
    columns = df.columns
    flatline_features = []
    for i in range(len(columns)):
        try:
            std = df[columns[i]].std()
            if std == 0:
                flatline_features.append(columns[i])
        except:
            pass
    #
    #print('Features which are considered flatline:\n')
    #for col in flatline_features:
    #    print(col)
    print('\nShape before changes: [' + str(df.shape) + ']')
    df = df.drop(columns=flatline_features)
    print('Shape after changes: [' + str(df.shape) + ']')
    print('Dropped a total [' + str(len(flatline_features)) + ']')
    return df
#
df = drop_flatline_columns(df=df)
print('\nAfter flatline column drop:')
print(df.shape)
print(df.columns)

### Data Ordering

Sorting of datasets in order of 
* SNAP_ID
* SQL_ID
* ID

In [ ]:
df.sort_values(by=['TIMESTAMP','SQL_ID','ID'], ascending=True, inplace=True)
print(df.head())

### Label Encoding

Converting labels/features into numerical representations

In [ ]:
def encode(df, encoded_labels):
    for col in df.columns:
        if col in encoded_labels:
            le = preprocessing.LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
    return df
#
#print(rep_vsql_plan_df.head())
encoded_labels = ['OPERATION','OPTIONS','OBJECT_OWNER','OBJECT_NAME','OBJECT_ALIAS','OBJECT_TYPE','OPTIMIZER','QBLOCK_NAME']
df = encode(df=df, encoded_labels=encoded_labels)
print('Encoded labels:\n' + str(encoded_labels) + "\n\n----------------------------------------------\n\n")
print(df.head())

### Access Plan Resource Aggregation

This method attempts to tackle the problem of access plan anomolies by aggregating resources per explain plan. Notable resources which are being considered are as follows:

* COST
* CARDINALITY
* BYTES
* PARTITION_DELTA (Partition End - Partition Start)
* CPU_COST
* IO_COST
* TEMP_SPACE
* TIME

The reasoning behind these fields in particular is mainly because these columns can be aggregated together.

In [ ]:
df_aggregate = df.groupby(['SQL_TEXT','PLAN_HASH_VALUE']).sum()
df_aggregate.reset_index(inplace=True)
print(df_aggregate.columns)
print(df_aggregate.shape)
for index, row in df_aggregate.iterrows():
    print(row['SQL_TEXT'])